#### What should the farmer do?

#### 0. Importing libraries

In [3]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
%pip install numpy
import numpy as np

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Part 1: Scenario Problems

### Case 1: Average Yields

#### 1. Set up the model

In [3]:
burrito_model = gp.Model()

Restricted license - for non-production use only - expires 2023-10-25


#### 2. Define sets and parameters

In [4]:
I = [1, 2] # list of customers
J = [1, 2, 3] # list of potential truck locations

d = [50, 40] # demand
c = np.array([[.1, 3, 5],
              [5, 3, .1]]) # distance from each customer to each potential truck location
f = 250 # setup cost per truck
r = 10 # unit burrito revenue
k = 5 # unit burrito ingredient cost

#### 3. Set the variables and the objective function

In [5]:
x = burrito_model.addVars(J, vtype = GRB.BINARY, name = "x")
y = burrito_model.addVars(I, J, vtype = GRB.BINARY, name = "y")

In [6]:
y

{(1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>}

In [7]:
burrito_model.setObjective(gp.quicksum(gp.quicksum((r - k) * 1/(c[i - 1, j - 1]) * d[i - 1] * y[i, j] for j in J) for i in I) - gp.quicksum(f * x[j] for j in J))

#### 4. Set the "sense" of the optimization problem

In [8]:
burrito_model.ModelSense = GRB.MAXIMIZE

#### 5. Add the constraints

In [9]:
for i in I:
    burrito_model.addConstr(gp.quicksum(y[i, j] for j in J) <= 1)

for i in I:
    for j in J:
        burrito_model.addConstr(y[i, j] <= x[j])

#### 6. Update and optimize the model

In [10]:
burrito_model.update()
burrito_model.write('burrito_truck_location.lp')
burrito_model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xd4cc80f6
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 8 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 4000 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+03, best bound 4.000000000000e+03, gap 0.0000%


#### 7. Print the results

In [11]:
print("The optimal objective value for this problem is {burrito_model.objVal}") # optimal objective value

print("The optimal values of the decision variables are as follows:")
for var in burrito_model.getVars():
    print(f"{var.varName} = {var.x}")

The optimal objective value for this problem is {burrito_model.objVal}
The optimal values of the decision variables are as follows:
x[1] = 1.0
x[2] = 0.0
x[3] = 1.0
y[1,1] = 1.0
y[1,2] = 0.0
y[1,3] = 0.0
y[2,1] = 0.0
y[2,2] = 0.0
y[2,3] = 1.0


### Case 2: Above Average Yields

### Case 3: Below Average Yields

# Part 2: Solving the Stochastic Problem (assuming that each yield scenario has an equal 1/3 probability of occurring)

In [5]:
# problem data
c = [150, 230, 260] # first stage objective coefficients
A = np.array([[1, 1, 1]]) # first stage constraint matrix
b = 500 # first stage right hand side
q = [238, 210, -170, -150, -36, -10] 
q = [1/3 * element for element in q] # second stage objective coefficients
T1 = np.array([[3, 0, 0], 
              [0, 3.6, 0],
              [0, 0, 24],
              [0, 0, 0],
              [0, 0, 0]]) # technology matrix for the first scenario
T2 = np.array([[2.5, 0, 0], 
              [0, 3, 0],
              [0, 0, 20],
              [0, 0, 0],
              [0, 0, 0]]) # technology matrix for the second scenario
T3 = np.array([[2, 0, 0], 
              [0, 2.4, 0],
              [0, 0, 16],
              [0, 0, 0],
              [0, 0, 0]]) # technology matrix for the third scenario
W = np.array([[1, 0, -1, 0, 0, 0], 
              [0, 1, 0, -1, 0, 0],
              [0, 0, 0, 0, -1, -1],
              [0, 0, 0, 0, -1, 0],
              [0, -1, 0, 0, 0, 0]]) # recourse matrix; the columns correspond to the second-stage variables in this order: y_1, y_2, w_1, w_2, w_3, and w_4
h = np.array([200, 240, 0, -6000, -40]) # second stage right-hand side
p = [1/3, 1/3, 1/3] # probability vector for the three scenarios


# suppress Gurobi output
environment = gp.Env(empty = True)
environment.setParam("OutputFlag", 0)
environment.start()


m = gp.Model('modified_farmers_problem_extensive_form', env = environment) # create model
x = m.addMVar(3, obj =  c, lb = 0.0, vtype = GRB.CONTINUOUS, name = "x") # define x variables and set objective values
y_1 = m.addMVar(6, obj =  q, lb = 0.0, vtype = GRB.CONTINUOUS, name = "y_1")
y_2 = m.addMVar(6, obj =  q, lb = 0.0, vtype = GRB.CONTINUOUS, name = "y_2")
y_3 = m.addMVar(6, obj =  q, lb = 0.0, vtype = GRB.CONTINUOUS, name = "y_3")
m.modelSense = GRB.MINIMIZE # set objective function sense
m.addConstr(A @ x <= b) # the farmer has 500 acres of land
m.addConstr(W @ y_1 >= h - T1 @ x)
m.addConstr(W @ y_2 >= h - T2 @ x)
m.addConstr(W @ y_3 >= h - T3 @ x)
m.update() # let Gurobi know that the model has changed
m.write("modified_farmers_problem_extensive_form.lp") # write out the lp in a lp-file
m.optimize() # optimize model

for v in m.getVars():
        if v.x >= 0:
            print(v.varName, v.x)

print('Optimal primal objective value : ', m.objVal) # display optimal value

x[0] 166.66666666666663
x[1] 83.33333333333334
x[2] 250.0
y_1[0] 0.0
y_1[1] 0.0
y_1[2] 299.9999999999999
y_1[3] 60.0
y_1[4] 6000.0
y_1[5] 0.0
y_2[0] 0.0
y_2[1] 0.0
y_2[2] 216.66666666666657
y_2[3] 10.0
y_2[4] 5000.0
y_2[5] 0.0
y_3[0] 0.0
y_3[1] 40.0
y_3[2] 133.33333333333326
y_3[3] 0.0
y_3[4] 4000.0
y_3[5] 0.0
Optimal primal objective value :  -108366.66666666667
